In [36]:
import albumentations as A
import cv2
import matplotlib.pyplot as plt
import os

from tqdm import tqdm

In [37]:
# Chemin du dossier qui contient les images et les bounding boxs
PATH = '3DPrintingAugmented'+"/"

In [38]:
# Chemins des dossiers qui contiendront les images et bounding boxs des images augmentées
# Ne pas oublier de créer ces Dossiers avant de lancer le code !
PATH_HORIZONTAL = PATH+'HorizontalFlip/'
PATH_VERTICAL = PATH+'VerticalFlip/'
PATH_TRANSPOSE = PATH+'Transpose/'
PATH_INVERT = PATH+'Invert/'
os.system("mkdir "+PATH_HORIZONTAL)
os.system("mkdir "+PATH_VERTICAL)
os.system("mkdir "+PATH_TRANSPOSE)
os.system("mkdir "+PATH_INVERT)

0

In [39]:
# Fonctions pour afficher les images avec les bounding boxs

BOX_COLOR = (255, 0, 0)  # Red
TEXT_COLOR = (255, 255, 255)  # White


def visualize_bbox(img, bbox, class_name, color=BOX_COLOR, thickness=2):
    """Visualizes a single bounding box on the image"""
    x_min, y_min, w, h = bbox
    x_min, x_max, y_min, y_max = int(x_min), int(
        x_min + w), int(y_min), int(y_min + h)

    cv2.rectangle(img, (x_min, y_min), (x_max, y_max),
                  color=color, thickness=thickness)

    ((text_width, text_height), _) = cv2.getTextSize(
        class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)
    cv2.rectangle(img, (x_min, y_min - int(1.3 * text_height)),
                  (x_min + text_width, y_min), BOX_COLOR, -1)
    cv2.putText(
        img,
        text=class_name,
        org=(x_min, y_min - int(0.3 * text_height)),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=0.35,
        color=TEXT_COLOR,
        lineType=cv2.LINE_AA,
    )
    return img


def visualize(image, bboxes, category_ids, category_id_to_name):
    img = image.copy()
    for bbox, category_id in zip(bboxes, category_ids):
        class_name = category_id_to_name[category_id]
        img = visualize_bbox(img, bbox, class_name)
    plt.figure(figsize=(12, 12))
    plt.axis('off')
    plt.imshow(img)

In [40]:
def get_bboxes(label_file):
    f = open(label_file)
    bboxes = f.readlines()
    bboxes = [bbox.replace('\n', '').split(' ')[1:] for bbox in bboxes]
    bboxes = [[float(i) for i in bbox] for bbox in bboxes]
    return bboxes

In [41]:
def save_new_bboxes(bboxes, labels, path):
    textfile = [(i, *j) for i, j in zip(labels, bboxes)]
    textfile = [' '.join(str(i) for i in bbox) + '\n' for bbox in textfile]
    nf = open(path, 'w')
    nf.writelines(textfile)
    nf.close()

In [42]:
def flip_augmentations(path_to_imgs=PATH, path_horizontal=PATH_HORIZONTAL, path_vertical=PATH_VERTICAL, path_transpose=PATH_TRANSPOSE):
    all_files = os.listdir(path_to_imgs)
    image_files = [file for file in all_files if file.endswith(
        '.jpeg') or file.endswith('.jpg')]
    label_files = [file for file in all_files if file.endswith('.txt')]

    horizontal_flip = A.Compose([
        A.HorizontalFlip(p=1)
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

    vertical_flip = A.Compose([
        A.VerticalFlip(p=1)
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

    transpose = A.Compose([
        A.Transpose(p=1)
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

    for img in tqdm(image_files):
        label_file = img.split('.')[0]
        if label_file + '.txt' in label_files:
            image = cv2.imread(path_to_imgs + img)
            bboxes = get_bboxes(path_to_imgs + label_file + '.txt')
            labels = [0] * len(bboxes)
            h = horizontal_flip(image=image, bboxes=bboxes,
                                class_labels=labels)
            v = vertical_flip(image=image, bboxes=bboxes, class_labels=labels)
            t = transpose(image=image, bboxes=bboxes, class_labels=labels)
            img_h, bbox_h, labels_h = h['image'], h['bboxes'], h['class_labels']
            img_v, bbox_v, labels_v = v['image'], v['bboxes'], v['class_labels']
            img_t, bbox_t, labels_t = t['image'], t['bboxes'], t['class_labels']
            save_new_bboxes(bbox_h, labels_h,
                            path_horizontal + f'{label_file}_H.txt')
            cv2.imwrite(path_horizontal + f'{label_file}_H.jpeg', img_h)
            save_new_bboxes(bbox_v, labels_v, path_vertical +
                            f'{label_file}_V.txt')
            cv2.imwrite(path_vertical + f'{label_file}_V.jpeg', img_v)
            save_new_bboxes(bbox_t, labels_t, path_transpose +
                            f'{label_file}_T.txt')
            cv2.imwrite(path_transpose + f'{label_file}_T.jpeg', img_t)

In [43]:
def invert_augmentation(path_to_imgs=PATH, path_horizontal=PATH_HORIZONTAL, path_vertical=PATH_VERTICAL, path_transpose=PATH_TRANSPOSE, path_invert=PATH_INVERT):
    base_files = os.listdir(path_to_imgs)
    H_files = os.listdir(path_horizontal)
    V_files = os.listdir(path_vertical)
    T_files = os.listdir(path_transpose)
    
    image_base_files = [file for file in base_files if file.endswith('.jpeg') or file.endswith('.jpg')]
    label_base_files = [file for file in base_files if file.endswith('.txt')]
    
    image_H_files = [file for file in H_files if file.endswith('.jpeg') or file.endswith('.jpg')]
#     label_H_files = [file for file in H_files if file.endswith('.txt')]
    image_V_files = [file for file in V_files if file.endswith('.jpeg') or file.endswith('.jpg')]
    image_T_files = [file for file in T_files if file.endswith('.jpeg') or file.endswith('.jpg')]
    
    invert_img = A.Compose([
        A.InvertImg(p=1)
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))
    
    print('Images de base en cours...')
    for img in tqdm(image_base_files):
        label_file = img.split('.')[0]
        if label_file + '.txt' in label_base_files:
            image = cv2.imread(path_to_imgs + img)
            bboxes = get_bboxes(path_to_imgs + label_file + '.txt')
            labels = [0] * len(bboxes)
            i = invert_img(image=image, bboxes=bboxes, class_labels=labels)
            img_i, bbox_i, labels_i = i['image'], i['bboxes'], i['class_labels']
            save_new_bboxes(bbox_i, labels_i, path_invert + f'{label_file}_I.txt')
            cv2.imwrite(path_invert + f'{label_file}_I.jpeg', img_i)
            
    print('Images retournées horizontalement en cours...')
    for img in tqdm(image_H_files):
        label_file = img.split('.')[0]
        image = cv2.imread(path_horizontal + img)
        bboxes = get_bboxes(path_horizontal + label_file + '.txt')
        labels = [0] * len(bboxes)
        i = invert_img(image=image, bboxes=bboxes, class_labels=labels)
        img_i, bbox_i, labels_i = i['image'], i['bboxes'], i['class_labels']
        save_new_bboxes(bbox_i, labels_i, path_invert + f'{label_file}_I.txt')
        cv2.imwrite(path_invert + f'{label_file}_I.jpeg', img_i)
    
    print('Images retournées verticalement en cours...')
    for img in tqdm(image_V_files):
        label_file = img.split('.')[0]
        image = cv2.imread(path_vertical + img)
        bboxes = get_bboxes(path_vertical + label_file + '.txt')
        labels = [0] * len(bboxes)
        i = invert_img(image=image, bboxes=bboxes, class_labels=labels)
        img_i, bbox_i, labels_i = i['image'], i['bboxes'], i['class_labels']
        save_new_bboxes(bbox_i, labels_i, path_invert + f'{label_file}_I.txt')
        cv2.imwrite(path_invert + f'{label_file}_I.jpeg', img_i)
    
    print('Images transposées en cours...')
    for img in tqdm(image_T_files):
        label_file = img.split('.')[0]
        image = cv2.imread(path_transpose + img)
        bboxes = get_bboxes(path_transpose + label_file + '.txt')
        labels = [0] * len(bboxes)
        i = invert_img(image=image, bboxes=bboxes, class_labels=labels)
        img_i, bbox_i, labels_i = i['image'], i['bboxes'], i['class_labels']
        save_new_bboxes(bbox_i, labels_i, path_invert + f'{label_file}_I.txt')
        cv2.imwrite(path_invert + f'{label_file}_I.jpeg', img_i)
    
    print('Fini !')

In [44]:
# C'est parti !
flip_augmentations()

100%|██████████| 340/340 [00:02<00:00, 116.55it/s]


In [45]:
# Nouvelle augmentation de toutes les images crées cette fois ci !
invert_augmentation()

Images de base en cours...


100%|██████████| 340/340 [00:01<00:00, 267.47it/s]


Images retournées horizontalement en cours...


100%|██████████| 340/340 [00:01<00:00, 311.59it/s]


Images retournées verticalement en cours...


100%|██████████| 340/340 [00:01<00:00, 291.19it/s]


Images transposées en cours...


100%|██████████| 340/340 [00:01<00:00, 214.70it/s]

Fini !
